In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

In [3]:
nxseed = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
np.random.seed(5)
torch.manual_seed(5)
if device is not "cpu":
    torch.cuda.manual_seed(5)

In [4]:
def generate_configurational(graphs):
    configurational_graphs = []
    for graphs in graph:
        configurational_graphs.append(configuration_model())
        

In [6]:
graphdatabase = GraphData(2500, 50)
graphdatabase.generate_BAGraphs()
# graphdatabase.generate_ERGraphs()
# graphdatabase.generate_adjs()
# graphdatabase.normalise()
# graphdatabase.tokenise()

In [9]:
graph = graphdatabase.graphs[0]

In [11]:
graph.nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49))

In [27]:
degree = list(dict(graph.degree()).values())

In [30]:
graph.degree()

DegreeView({0: 13, 1: 3, 2: 5, 3: 3, 4: 2, 5: 1, 6: 4, 7: 2, 8: 16, 9: 3, 10: 2, 11: 1, 12: 2, 13: 1, 14: 1, 15: 3, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 3, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1})

In [29]:
nx.configuration_model(degree).degree()

MultiDegreeView({0: 13, 1: 3, 2: 5, 3: 3, 4: 2, 5: 1, 6: 4, 7: 2, 8: 16, 9: 3, 10: 2, 11: 1, 12: 2, 13: 1, 14: 1, 15: 3, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 3, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1})

In [33]:
nx.adjacency_matrix(graph).todense()

matrix([[0, 1, 1, ..., 0, 1, 1],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [35]:
nx.adjacency_matrix(nx.configuration_model(degree)).todense()

matrix([[1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [9]:
for graphs in graph:
    configurational_graphs.append(configuration_model())
    graphdatabase.graphs

 ...]

In [5]:
class GraphData():
    def __init__(self, num, nodes):
        self.num = num
        self.nodes = nodes
        self.m_list = [1.0, nodes/8.0, nodes/4.0, 3.0*nodes/8.0, nodes/2.0]
        self.p_list = np.linspace(1.0/num, num/2.0, 4)
        self.graphs = []
        self.graphadjs = []
        self.graphlabels = []
        self.tokenisedgraphlabels = []
        self.graphadjs_da = []
        self.graphadjs_dad = []
        
    def generate_BAGraphs(self):
        num_each_group = int(self.num/len(self.m_list))
        
        for m in self.m_list:
            for i in range(num_each_group):
                graph = nx.barabasi_albert_graph(self.nodes, int(m), nxseed)
                self.graphs.append(graph)
                self.graphlabels.append("BA")
                
    def generate_ERGraphs(self):
        num_each_group = int(self.num/len(self.p_list))
        
        for p in self.p_list:
            for i in range(num_each_group):
                graph = nx.erdos_renyi_graph(self.nodes, p, nxseed, directed=False)
                self.graphs.append(graph)
                self.graphlabels.append("ER")
                      
    def generate_adjs(self):
        for graph in self.graphs:
            self.graphadjs.append(nx.adjacency_matrix(graph).todense())
    def normalise(self):
        for x in self.graphadjs:
            rowsum = np.array(x.sum(1), dtype = np.float32)
            r_inv = np.power(rowsum, -1).flatten()
            r_inv[np.isinf(r_inv)] = 0.
            d = np.diag(r_inv)
            da = d.dot(x)
            d = np.sqrt(d)
            dad = (d.dot(x)).dot(d)
            self.graphadjs_da.append(da)
            self.graphadjs_dad.append(dad)
    def tokenise(self):
        self.tokenisedgraphlabels = [1 if x=="BA" else 0 for x in self.graphlabels]

In [ ]:
r = argsort(rand(len(a)))
Adj_nodes_shuffled[i] = a[r][:,r]

In [9]:
for i, a in enumerate(graphdatabase.graphadjs):
    

(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)
(50, 50)


In [ ]:
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, h, y):
        model.train()
        yhat = model(h, x)      
        loss = loss_fn(y, yhat)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

In [ ]:
def make_loader(x_h, y, batch_size, ratio_train):

    x_tensor = torch.from_numpy(x_h[0]).float()
    h_tensor = torch.from_numpy(x_h[1]).float()
    y_tensor = torch.from_numpy(y).float()

    dataset = TensorDataset(x_tensor, h_tensor, y_tensor)
    trainlength = int(ratio_train*x_h[0].shape[0])
    vallength = x_h[0].shape[0] - trainlength
    
    train_dataset, val_dataset = random_split(dataset, [trainlength, vallength])
    train_loader = DataLoader(dataset=train_dataset, batch_size = batch_size)
    val_loader = DataLoader(dataset=val_dataset, batch_size = batch_size)
    return train_loader, val_loader

In [ ]:
def train_model(model, device, train_step, n_epochs, train_loader, val_loader):
    training_losses = []
    validation_losses = []

    for epoch in range(n_epochs):
        batch_losses = []
        for x_batch, h_batch, y_batch in train_loader:
            x_batch = x_batch.to(device)
            h_batch = h_batch.to(device)
            y_batch = y_batch.to(device)
            loss = train_step(x_batch, h_batch, y_batch)
            batch_losses.append(loss)
        training_loss = np.mean(batch_losses)
        training_losses.append(training_loss)

        with torch.no_grad():
            val_losses = []
            for x_val, h_val, y_val in val_loader:
                x_val = x_val.to(device)
                h_val = h_val.to(device)
                y_val = y_val.to(device)
                model.eval()
                
                yhat = model(h_val, x_val)
                print(y_val.shape)
                print(yhat.shape)
                yhat = (yhat >.5).float()
                val_loss = (y_val == yhat).float().sum()/(y_val.shape[0])
#                 print("Val loss is:", val_loss.item())
                val_losses.append(val_loss.item())
#                 print(dir(val_loss))
            validation_loss = np.mean(val_losses)
            validation_losses.append(validation_loss)

        print(f"[{epoch+1}] Training loss: {training_loss:.3f}\t Validation loss: {validation_loss:.3f}")
    return training_losses, validation_losses

In [6]:
graphdatabase = GraphData(2500, 50)
graphdatabase.generate_BAGraphs()
graphdatabase.generate_ERGraphs()
graphdatabase.generate_adjs()
graphdatabase.normalise()
graphdatabase.tokenise()

c:\users\hr21924\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in power


In [7]:
len(graphdatabase.graphlabels)
len(graphdatabase.graphadjs)

5000

In [265]:
class GCN(nn.Module):
    def __init__(self, nnodes, graphoutputfeat):
        super(GCN, self).__init__()
        self.fc = nn.Linear(in_features=nnodes, out_features=graphoutputfeat)
    def forward(self, input, adj):
        x = F.relu(torch.bmm(adj, input))
        x = x.view(-1, x.shape[-2])
        x = self.fc(x)
        return x

In [266]:
X = np.array(graphdatabase.graphadjs_dad)
Y = np.expand_dims(np.array(graphdatabase.tokenisedgraphlabels), axis = 1)
h = np.ones(X.shape[:2]+(1,))

In [267]:
print(X.shape)
print(Y.shape)
print(h.shape)

(5000, 50, 50)
(5000, 1)
(5000, 50, 1)


In [268]:
net = GCN(h.shape[-2], 1).to(device)

In [269]:
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(net.parameters(), lr=.001)
train_step = make_train_step(net, loss_fn, optimizer)

In [270]:
train_loader, val_loader = make_loader([X,h], Y, int(5000), .2)
training_losses, validation_losses = train_model(net, device, train_step, 200, train_loader, val_loader)
validation_loss = min(validation_losses)
training_loss = training_losses[np.argmin(validation_loss)]
# histories[n_samples] = {"loss": training_losses, "val_loss" :validation_losses}

torch.Size([4000, 1])
torch.Size([4000, 1])
[1] Training loss: 0.223	 Validation loss: 0.698
torch.Size([4000, 1])
torch.Size([4000, 1])
[2] Training loss: 0.212	 Validation loss: 0.698
torch.Size([4000, 1])
torch.Size([4000, 1])
[3] Training loss: 0.206	 Validation loss: 0.799
torch.Size([4000, 1])
torch.Size([4000, 1])
[4] Training loss: 0.203	 Validation loss: 0.529
torch.Size([4000, 1])
torch.Size([4000, 1])
[5] Training loss: 0.203	 Validation loss: 0.629
torch.Size([4000, 1])
torch.Size([4000, 1])
[6] Training loss: 0.204	 Validation loss: 0.629
torch.Size([4000, 1])
torch.Size([4000, 1])
[7] Training loss: 0.204	 Validation loss: 0.629
torch.Size([4000, 1])
torch.Size([4000, 1])
[8] Training loss: 0.203	 Validation loss: 0.629
torch.Size([4000, 1])
torch.Size([4000, 1])
[9] Training loss: 0.202	 Validation loss: 0.629
torch.Size([4000, 1])
torch.Size([4000, 1])
[10] Training loss: 0.200	 Validation loss: 0.629
torch.Size([4000, 1])
torch.Size([4000, 1])
[11] Training loss: 0.197

[174] Training loss: 0.064	 Validation loss: 1.000
torch.Size([4000, 1])
torch.Size([4000, 1])
[175] Training loss: 0.064	 Validation loss: 1.000
torch.Size([4000, 1])
torch.Size([4000, 1])
[176] Training loss: 0.063	 Validation loss: 1.000
torch.Size([4000, 1])
torch.Size([4000, 1])
[177] Training loss: 0.063	 Validation loss: 1.000
torch.Size([4000, 1])
torch.Size([4000, 1])
[178] Training loss: 0.062	 Validation loss: 1.000
torch.Size([4000, 1])
torch.Size([4000, 1])
[179] Training loss: 0.062	 Validation loss: 1.000
torch.Size([4000, 1])
torch.Size([4000, 1])
[180] Training loss: 0.061	 Validation loss: 1.000
torch.Size([4000, 1])
torch.Size([4000, 1])
[181] Training loss: 0.061	 Validation loss: 1.000
torch.Size([4000, 1])
torch.Size([4000, 1])
[182] Training loss: 0.060	 Validation loss: 1.000
torch.Size([4000, 1])
torch.Size([4000, 1])
[183] Training loss: 0.060	 Validation loss: 1.000
torch.Size([4000, 1])
torch.Size([4000, 1])
[184] Training loss: 0.060	 Validation loss: 1.000